# Яндекс Музыка

В данном проекте выполняется сравнение поведения пользователей Яндекс Музыки в двух городах Москве и Санкт-Петербурге. 

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

В обзоре данных, перед проверкой гипотез, мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы найдём возможность исправить самые критичные ошибки данных.

Таким образом, исследование проходит в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

**Описание колонок:**
userID — идентификатор пользователя;
Track — название трека;
artist — имя исполнителя;
genre — название жанра;
City — город пользователя;
time — время начала прослушивания;
Day — день недели.

## Обзор данных

Составим первое представление о данных Яндекс Музыки.

Импортируем библиотеку — `pandas`:

In [125]:
import pandas as pd # импорт библиотеки pandas

Прочитаем — `datasets` и сохраним его в переменной — `df`:

In [126]:
df = pd.read_csv('yandex_music_dataset.csv') # чтение файла с данными и сохранение в df

Выведем на экран первые десять строк таблицы:

In [ ]:
print(df.head(10)) # получение первых 10 строк таблицы df

| user_id | track | artist           | genre | city | time | day |
|---|---|------------------|---|---|---|---|
| FFB692EC | Kamigata To Boots | The Mass Missile | rock | Saint-Petersburg | 20:28:33 | Wednesday |
| 55204538 | Delayed Because of Accident | Andreas Rönnberg | rock | Moscow | 14:07:09 | Friday |
| 20EC38 | Funiculì funiculà | Mario Lanza      | pop | Saint-Petersburg | 20:58:07 | Wednesday |
| A3DD03C9 | Dragons in the Sunset | Fire + Ice       | folk | Saint-Petersburg | 08:37:09 | Monday |
| E2DC1FAE | Soul People | Space Echo       | dance | Moscow | 08:34:34 | Monday |
| 842029A1 | Преданная | IMPERVTOR        | rusrap | Saint-Petersburg | 13:09:41 | Friday |
| 4CB90AA5 | True | Roman Messer     | dance            | Moscow | 13:00:07 | Wednesday |
| F03E1C1F | Feeling This Way | Polina Griffith  | dance | Moscow | 20:47:49 | Wednesday |
| 8FA1D3BE | И вновь продолжается бой | unknown          | ruspop | Moscow | 09:17:40 | Friday |
| E772D5C0 | Pessimist | unknown          | dance | Saint-Petersburg | 21:20:49 | Wednesday |


Получим общую информацию о таблице c помощью метода — `info()`:

In [ ]:
df.info() # получение общей информации о данных в таблице df

| | |
| --- | --- |
| **Column** | **Non-Null Count** | **Dtype** |
| --- | --- | --- |
| user_id | 61253 | object |
| track | 61253 | object |
| artist | 61253 | object |
| genre | 61253 | object |
| city | 61253 | object |
| time | 61253 | object |
| day | 61253 | object |
| --- | --- | --- |
| **dtypes:** object(7) | **memory usage:** 3.7+ MB |

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с легко читаемым стилем.

Чтобы двигаться дальше, устраним проблемы в данных.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков

Выведем на экран названия столбцов:

In [ ]:
print(df.columns) # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведём названия к легко читаемому стилю:
* запишем в «змеином_регистре»;
* все символы сделаем строчными;
* устраним пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [130]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}) 
# переименование столбцов

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [ ]:
df.columns # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

Сначала посчитаем, сколько в таблице пропущенных значений. Для этого достаточно двух методов — `pandas`:

In [ ]:
df.isna().sum() # подсчёт пропусков

|  | &lt;unnamed&gt; |
| :--- | :--- |
| user\_id | 0 |
| track | 1231 |
| artist | 7203 |
| genre | 1198 |
| city | 0 |
| time | 0 |
| day | 0 |

Не все пропущенные значения влияют на исследование. Так, в `track` и `artist` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнить музыкальные вкусы в Москве и Санкт-Петербурге. В другом проекте было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в данном проекте. Поэтому:
* заполним и эти пропуски явными обозначениями;
* оценим, насколько они повредят расчётам.

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберём его элементы циклом — `for` и для каждого столбца выполним замену пропущенных значений:

In [133]:
columns_to_replace = ['track', 'artist', 'genre']
for columns in columns_to_replace:
    df[columns] = df[columns].fillna('unknown') 
    # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [ ]:
df.isna().sum() # подсчёт пропусков

|  | &lt;unnamed&gt; |
| :--- | :--- |
| user\_id | 0 |
| track | 0 |
| artist | 0 |
| genre | 0 |
| city | 0 |
| time | 0 |
| day | 0 |

### Дубликаты

Посчитаем явные дубликаты в таблице одной командой:

In [ ]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

Вызовем специальный метод — `pandas`, чтобы удалить явные дубликаты:

In [136]:
df = df.drop_duplicates() # удаление явных дубликатов

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [ ]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь избавимся от неявных дубликатов в колонке — `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат нашего исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлекаем нужный столбец датафрейма;
* применим к нему метод сортировки;
* для отсортированного столбца вызовем метод, который вернёт уникальные значения из столбца.

In [ ]:
df['genre'].sort_values().unique() # Просмотр уникальных названий жанров

Просматриваем список и находим неявные дубликаты названия — `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Наблюдаем следующие неявные дубликаты:
* *hip*;
* *hop*;
* *hip-hop*.
Чтобы очистить от них таблицу, используем метод — `replace()` с двумя аргументами: списком строк-дубликатов (включающий *hip*, *hop* и *hip-hop*) и строкой с правильным значением. Исправим колонку `genre` в таблице `df`: заменим каждое значение из списка дубликатов на верное. Вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [139]:
df['genre'] = df['genre'].replace('hip', 'hiphop')
df['genre'] = df['genre'].replace('hop', 'hiphop')
df['genre'] = df['genre'].replace('hip-hop', 'hiphop') # Устранение неявных дубликатов

Проверим, что заменили неправильные названия:

*   *hip*;
*   *hop*;
*   *hip-hop*.

Выводим отсортированный список уникальных значений столбца — `genre`:

In [ ]:
df['genre'].sort_values().unique() # Проверка на неявные дубликаты

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков;
- пропущенные значения;
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на — `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке — `genre`.

Теперь переходим к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Выполняем каждый из расчётов по отдельности.

Оцениваем активность пользователей в каждом городе. Группируем данные по городу и считаем прослушивания в каждой группе.

In [ ]:
df.groupby('city')['user_id'].count() # Подсчёт прослушиваний в каждом городе

| <br/>city | user\_id<br/> |
| :--- | :--- |
| Moscow | 42741 |
| Saint-Petersburg | 18512 |

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь группируем данные по дню недели и считаем прослушивания в понедельник, среду и пятницу. Учтём, что в данных есть информация о прослушиваниях только за эти дни.

In [ ]:
df.groupby('day')['genre'].count() # Подсчёт прослушиваний в каждый из трёх дней

| <br/>day | genre<br/> |
| :--- | :--- |
| Friday | 21840 |
| Monday | 21354 |
| Wednesday | 18059 |

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создадим функцию — `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели;
* название города.

В функции сохраняем в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`;
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце — `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернём эту переменную из функции.

In [143]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list =  track_list[ track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return(track_list_count)
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id.
# Это количество функция вернёт в качестве результата.

Вызовем — `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [ ]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

15740

In [ ]:
number_tracks('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [ ]:
number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам

11056

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

7003

In [ ]:
number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам

15945

In [ ]:
number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам

5895

С помощью конструктора — `pd.DataFrame` создадим таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`.

In [150]:
data = [['Moscow', 15740, 11056, 15945],
        ['Saint-Petersburg', 5614, 7003, 5895]] 
columns = ['city', 'monday', 'wednesday', 'friday'] 
info = pd.DataFrame(data = data, columns = columns) # Таблица с результатами

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [151]:
moscow_general = df[df['city'] == 'Moscow'] # получение таблицы moscow_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Moscow'

In [152]:
spb_general = df[df['city'] == 'Saint-Petersburg'] # получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными;
* день недели;
* начальная временная метка в формате 'hh:mm';
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [153]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted =  genre_df_grouped.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Сравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 781 |
| dance | 549 |
| electronic | 480 |
| rock | 474 |
| hiphop | 286 |
| ruspop | 186 |
| world | 181 |
| rusrap | 175 |
| alternative | 164 |
| unknown | 161 |

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 218 |
| dance | 182 |
| rock | 162 |
| electronic | 147 |
| hiphop | 80 |
| ruspop | 64 |
| alternative | 58 |
| rusrap | 55 |
| jazz | 44 |
| classical | 40 |

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Москве

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 713 |
| rock | 517 |
| dance | 495 |
| electronic | 482 |
| hiphop | 273 |
| world | 208 |
| ruspop | 170 |
| alternative | 163 |
| classical | 163 |
| rusrap | 142 |

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Петербурге

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 256 |
| electronic | 216 |
| rock | 216 |
| dance | 210 |
| hiphop | 97 |
| alternative | 63 |
| jazz | 61 |
| classical | 60 |
| rusrap | 59 |
| world | 54 |

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение — `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу — `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом — `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице — `moscow_genres`.

In [158]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False) 
# одной строкой: группировка таблицы moscow_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

Выведем на экран первые десять строк — `moscow_genres`:

In [ ]:
moscow_genres.head(10) # просмотр первых 10 строк moscow_genres

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 5892 |
| dance | 4435 |
| rock | 3965 |
| electronic | 3786 |
| hiphop | 2096 |
| classical | 1616 |
| world | 1432 |
| alternative | 1379 |
| ruspop | 1372 |
| rusrap | 1161 |

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу — `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [160]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False) 
# одной строкой: группировка таблицы spb_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres

Выведем на экран первые десять строк — `spb_genres`:

In [ ]:
spb_genres.head(10) # просмотр первых 10 строк spb_genres

| <br/>genre | genre<br/> |
| :--- | :--- |
| pop | 2431 |
| dance | 1932 |
| rock | 1879 |
| electronic | 1736 |
| hiphop | 960 |
| alternative | 649 |
| classical | 646 |
| rusrap | 564 |
| ruspop | 538 |
| world | 515 |

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”;
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.